In [164]:
import pymysql
import requests,json
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt 
import datetime

# Sensor Tower

def rename_table(name_list, df_stpre):
    for (index, item) in enumerate(name_list):
        df_stpre[item] = df_stpre[index]     
    df_stpre = df_stpre.drop([i for i in range(len(name_list))], axis =1)
    return df_stpre
"""
category:
date 
country 
publisher_id 
platform
"""
def list_to_frame(my_df):
    df_aux = my_df.to_frame()
    df_aux = df_aux.reset_index()
    return df_aux


def get_publisher_df(df_preapps):
    aux_time, final_table = [], pd.DataFrame()
    for i in df_preapps['date']:
        if i not in aux_time:
            aux_time.append(i)
#     for index, item in enumerate(df_preapps['date']):    
#     print(item == aux_time[index])
    for i in aux_time:
        ios_table = df_preapps[(df_preapps['platform'] == 'ios') & (df_preapps['date'] == i)].groupby(['country', 'unified_id'])['downloads'].sum()
        android_table = df_preapps[(df_preapps['platform'] == 'android') & (df_preapps['date'] == i)].groupby(['country', 'unified_id'])['downloads'].sum()
        unified_table = df_preapps[df_preapps['date'] == i].groupby(['country', 'unified_id'])['downloads'].sum() # unified id 
        df_ios = list_to_frame(ios_table)
        df_android = list_to_frame(android_table)
        df_unified = list_to_frame(unified_table)
        
        df_ios['platform'] = pd.Series(['ios' for i in range(df_ios.shape[0])])
        df_android['platform'] = pd.Series(['android' for i in range(df_android.shape[0])])
        df_unified['platform'] = pd.Series(['unified' for i in range(df_unified.shape[0])]) 
        
        df_ios['start_date'] = pd.Series([ i for k in range(df_ios.shape[0])])
        df_android['start_date'] = pd.Series([i for k in range(df_android.shape[0])])
        df_unified['start_date'] = pd.Series([i for k in range(df_unified.shape[0])])
    
        df_ios['end_date'] = df_ios['start_date'].apply(lambda x: str(datetime.datetime.strptime(str(x), "%Y-%m-%d")+datetime.timedelta(days=6))[:10])
        df_android['end_date'] = df_android['start_date'].apply(lambda x: str(datetime.datetime.strptime(str(x), "%Y-%m-%d")+datetime.timedelta(days=6))[:10])
        df_unified['end_date'] = df_unified['start_date'].apply(lambda x: str(datetime.datetime.strptime(str(x), "%Y-%m-%d")+datetime.timedelta(days=6))[:10])
        final_table = pd.concat([final_table, df_ios])
        final_table = pd.concat([final_table, df_android])   
        final_table = pd.concat([final_table, df_unified])
        print(i)
    final_table = final_table.reset_index(drop = True)   
    final_table['source_platform'] = pd.Series(['Sensor Tower' for k in range(final_table.shape[0])])
    return final_table 

pd.set_option('display.max_rows', 5000)


In [87]:
conn = pymysql.connect(host='124.156.172.225', user='qianchen', passwd='qianchen123', db='market_bi')
cur=conn.cursor()
cur.execute('''
            SELECT * FROM premium_apps 
            '''
           )
test = cur.fetchall()
df_preapps = pd.DataFrame(list(test))

name_list = ['id', 'product_id', 'date', 'publisher_id', 'unified_id', 'platform', 'country', 'downloads', 'rating', 'update_date', 'source_platform']
df_preapps = rename_table(name_list, df_preapps)

In [93]:
# def select_top3_publisher(df_table):
#     final = pd.DataFrame()
#     aux = df_table.drop_duplicates(subset = ['country'], keep = 'first')
#     for i in aux['country']:
#         co_aux = df_table.loc[df_table['country'] == i]
#         or_co_aux = co_aux.sort_values(by = ['downloads'], ascending = False)
#         final = pd.concat([final, or_co_aux])
#     final.reset_index(drop = True)    
#     return final 

In [165]:
final = get_publisher_df(df_preapps)

2021-01-04
2021-01-11
2021-01-18
2021-01-25
2021-02-01
2021-02-08
2021-02-15
2021-02-22
2021-03-01
2021-03-08


In [166]:
final

,country,unified_id,downloads,platform,start_date,end_date,source_platform
0,AU,55c5017302ac64f9c0001f2d,3380,ios,2021-01-04,2021-01-10,Sensor Tower
1,AU,55c5022c02ac64f9c0001f84,12515,ios,2021-01-04,2021-01-10,Sensor Tower
2,AU,55c5029e02ac64f9c0001fc9,4141,ios,2021-01-04,2021-01-10,Sensor Tower
3,AU,55c505ed02ac64f9c0002113,2931,ios,2021-01-04,2021-01-10,Sensor Tower
4,AU,55d9183b0e53ae3c2a006091,1461,ios,2021-01-04,2021-01-10,Sensor Tower
...,...,...,...,...,...,...,...
31338,WW,5ff90781682b7c2f2ae5508d,2482889,unified,2021-03-08,2021-03-14,Sensor Tower
31339,WW,5ffdc380ec1eca0b6cd335da,604074,unified,2021-03-08,2021-03-14,Sensor Tower
31340,WW,5ffe4d72a25ee0595bdc08d3,488115,unified,2021-03-08,2021-03-14,Sensor Tower
31341,WW,602594e308a3d45a567d22f0,376822,unified,2021-03-08,2021-03-14,Sensor Tower
